# ===============================================
# 🏆 Project: The "Transaction-Shield" Swarm
# ===============================================
### **Track: Enterprise Agents 🛡️**


### **1. 🚀 The Pitch (Enterprise Agents Track)**

**The Problem:** E-commerce businesses lose billions annually to fraudulent transactions. Manually reviewing every order is impossible, but simplistic automated systems often block legitimate customers (false positives) or miss sophisticated fraud (false negatives).

**The Solution:** "Transaction-Shield" is an autonomous **enterprise agent swarm** that acts as a real-time fraud detection "daemon." It monitors a (mocked) live stream of new orders.

When a new order is detected, the swarm activates:
1.  A **Sequential** `MonitorAgent` logs the order.
2.  A **Parallel Team** of specialist agents (`HistoryAgent`, `BillingAgent`, `VelocityAgent`) investigate the order simultaneously for different "red flags."
3.  The agents **Read from Long-Term Memory** to check against known fraud lists and order-velocity history.
4.  The agents **Write to Long-Term Memory** to update the order-velocity history, making the swarm "smarter" with each transaction.
5.  An **Evaluation Agent** (the `RiskManagerAgent`) acts as the "brain." It reads all the technical reports, *calculates* a risk score, and *generates* a single, human-readable justification for its final decision: `APPROVE`, `REJECT`, or `FLAG FOR MANUAL REVIEW`.
6.  A final agent uses a **Real Custom Tool** to write all "flagged" orders to a `MANUAL_REVIEW_QUEUE.md` file for the human fraud-review team.

This project automates a high-stakes, real-world business workflow, saving the company money and protecting customer relationships.


### **2. ⚙️ Features Demonstrated**

This notebook is a capstone project that demonstrates all 6 key concepts:

1.  **Multi-agent System:**
    * **Sequential Agents:** `MonitorAgent` -> (Parallel Team) -> `RiskManagerAgent` -> `PublisherAgent`.
    * **Parallel Agents:** `HistoryAgent`, `BillingAgent`, and `VelocityAgent` all run at the same time.
    * **Loop Agent:** The main `run_transaction_shield_swarm` function acts as a "daemon" loop, processing a queue of transactions.

2.  **Tools:**
    * **Mocked Built-in Tool:** `tool_get_customer_history` is used by the `HistoryAgent` (simulates a database lookup).
    * **Mocked Built-in Tool:** `tool_check_ip_location` is used by the `BillingAgent` (simulates an IP-to-location API).
    * **Real Custom Tool:** `tool_flag_for_manual_review` saves the final, formatted alert to a `.md` file.

3.  **Long-Running Operations (Pause/Resume):**
    * The main swarm function is a **Loop** that processes the order queue and then "pauses" (`time.sleep`) to wait for new transactions.

4.  **Long-Term Memory:**
    * **Write:** The `VelocityAgent` **writes** timestamps to `MEMORY_BANK["order_timestamps_by_ip"]` *after every order*.
    * **Read:** The `VelocityAgent` **reads** this same list to detect high-velocity fraud. The `HistoryAgent` **reads** `MEMORY_BANK["known_fraud_users"]`. This is a very advanced, stateful memory system.

5.  **Agent Evaluation (The "Brain"):**
    * The `RiskManagerAgent` is the "brain." It reads and evaluates three different technical reports, *calculates* a `risk_score`, and *generates* a new, human-readable `justification` string for its final decision.

6.  **Observability:**
    * A `log_message()` function provides a rich, real-time log of every agent's actions and decisions.
    * 

### **3. 🤖 System Architecture**

This swarm operates on a Sequential -> Parallel -> Sequential workflow.

Here is the step-by-step data and logic flow:

1.  **[LOOP START]** (`run_transaction_shield_swarm`)
    * `MonitorAgent` pulls one `order` from the (mocked) queue.

2.  **[PARALLEL TEAM]** (Specialists)
    * The `MonitorAgent` passes the `order` to three specialists who run at the *same time*:
        * `HistoryAgent`: Checks `customer_id` against `MEMORY_BANK["known_fraud_users"]` (Memory **Read**).
        * `BillingAgent`: Checks if `billing_country` != `shipping_country`.
        * `VelocityAgent`: Checks `ip_address` against `MEMORY_BANK["order_timestamps_by_ip"]` (Memory **Read**), then **writes** the new timestamp to it (Memory **Write**).

3.  **[SEQUENTIAL 2]** (`RiskManagerAgent` - The "Brain")
    * Waits for all three parallel agents to finish.
    * **Evaluates** their reports, calculates a final `risk_score`.
    * **Generates** a `justification` string.
    * **Writes** this finding to `MEMORY_BANK["manual_review_queue"]` (Memory **Write**).

4.  **[SEQUENTIAL 3]** (`PublisherAgent`)
    * **If** the "Brain's" decision is `FLAG_FOR_REVIEW`, this agent uses the **Custom Tool** (`tool_flag_for_manual_review`) to save the justification to `MANUAL_REVIEW_QUEUE.md`.

5.  **[LOOP END]**
    * The swarm **pauses** (`time.sleep`) and waits for the next order.
    * 

### **4. 📚 Table of Contents**

* **Cell 1:** Imports & Libraries
* **Cell 2:** Global Setup (Memory Bank & Logger)
* **Cell 3:** Pre-load Memory (Known Fraudsters)
* **Cell 4:** Agent Toolbox (Mocked & Custom Tools)
* **Cell 5:** The Agent Team: Part 1 (Specialists)
* **Cell 6:** The Agent Team: Part 2 (The "Brain")
* **Cell 7:** The "Daemon" Loop (Main Function)
* **Cell 8:** 🚀 **Run the Swarm!** 🚀
* **Cell 9:** Review: Long-Term Memory
* **Cell 10:** 🔔 **Review: The Final "Flagged" File** 🔔
* 

# 📚 Imports & Libraries

In [1]:
import time
import json
import datetime
import random
import os
import textwrap # Used for formatting our final Markdown report
from collections import defaultdict

print("Cell 1: Libraries imported successfully.")


Cell 1: Libraries imported successfully.


# 🧠 Global Setup (Memory Bank & Logger)

In [2]:
MEMORY_BANK = {
    "known_fraud_users": set(),
    # This dict will hold: {"ip_address": [timestamp1, timestamp2, ...]}
    "order_timestamps_by_ip": defaultdict(list),
    # This list will hold the "Brain's" audit log
    "decision_log": [],
    "alerts_fired": 0,
}

# This helper function will act as our Observability/Logging
def log_message(agent_name, message, level="INFO"):
    """Helper function for formatted logging."""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] [{agent_name}] [{level}]: {message}")

print("Cell 2: Global Memory Bank and Logger initialized.")


Cell 2: Global Memory Bank and Logger initialized.


# 💾 Pre-load Memory (Known Fraudsters)
We pre-load the Long-Term Memory with known bad actors for the HistoryAgent to read from.

In [3]:
MEMORY_BANK["known_fraud_users"].add("user_12345")
MEMORY_BANK["known_fraud_users"].add("user_99876")

print("Cell 3: Long-Term Memory pre-loaded with 'Known Fraudsters'.")
print(f"Known fraud users: {MEMORY_BANK['known_fraud_users']}")


Cell 3: Long-Term Memory pre-loaded with 'Known Fraudsters'.
Known fraud users: {'user_99876', 'user_12345'}


# 🛠️ Agent Toolbox (Mocked & Custom Tools

In [4]:
# --- Mocked Built-in Tool (Database Lookup) ---
def tool_get_customer_history(customer_id: str) -> str:
    """MOCK BUILT-IN TOOL: Simulates a database lookup for customer history."""
    log_message("CustomerDBTool", f"Looking up history for: {customer_id}")
    
    # Mock database
    db = {
        "user_77788": {"order_count": 25, "total_spent_usd": 1200.50, "is_vip": True},
        "user_00001": {"order_count": 1, "total_spent_usd": 15.00, "is_vip": False}
    }
    
    # Return history or a "new customer" record
    history = db.get(customer_id, {"order_count": 0, "total_spent_usd": 0, "is_vip": False})
    return json.dumps(history)

# --- Mocked Built-in Tool (Geo-IP Lookup) ---
def tool_check_ip_location(ip_address: str) -> str:
    """MOCK BUILT-IN TOOL: Simulates checking the country of an IP address."""
    log_message("GeoIPTool", f"Checking location for IP: {ip_address}")
    
    # Mock database
    ip_db = {
        "1.2.3.4": "USA",
        "5.6.7.8": "USA",
        "9.8.7.6": "NIGERIA" # This will be a mismatch
    }
    country = ip_db.get(ip_address, "UNKNOWN")
    return json.dumps({"country": country})

# --- REAL Custom Tool (Markdown File Saver) ---
def tool_flag_for_manual_review(order_id: int, justification: str, filename: str):
    """
    REAL CUSTOM TOOL: Saves the final, formatted alert to a Markdown file
    for the human review team.
    """
    log_message("CustomFileTool", f"Saving alert for Order {order_id} to '{filename}'")
    try:
        # textwrap.dedent removes leading whitespace, making our report clean
        report = textwrap.dedent(f"""
        -------------------------------------------
        ###  flagged FOR MANUAL REVIEW
        **Order ID:** {order_id}
        **Justification:** {justification}
        -------------------------------------------
        """)
        
        with open(filename, 'a') as f:
            f.write(report + "\n")
            
        log_message("CustomFileTool", "Alert successfully saved.")
        MEMORY_BANK["alerts_fired"] += 1
    except Exception as e:
        log_message("CustomFileTool", f"Failed to save file: {e}", level="ERROR")

print("Cell 4: All tools (mocked and custom) are defined.")


Cell 4: All tools (mocked and custom) are defined.


# 🤖 The Agent Team: Part 1 (The Specialists)
These are the parallel agents who perform the first-pass investigation

In [5]:
# --- Sequential Agent 1: The Monitor ---
def monitor_agent(order: dict) -> dict:
    """SEQUENTIAL AGENT 1: Logs the new order and passes it to the team."""
    log_message("MonitorAgent", f"New order detected: {order['order_id']}. Passing to specialists.")
    return {"order_data": order}

# --- Parallel Agent 1: The History Agent ---
def history_agent(order_data: dict) -> dict:
    """PARALLEL AGENT 1: Checks customer history and known fraud lists."""
    log_message("HistoryAgent", f"Checking history for user {order_data['customer_id']}...")
    flags = []
    
    # Use the (mocked) database tool
    history = json.loads(tool_get_customer_history(order_data['customer_id']))
    
    if history["order_count"] == 0:
        flags.append("New customer (higher risk)")
        
    # READS from Long-Term Memory
    if order_data["customer_id"] in MEMORY_BANK["known_fraud_users"]:
        flags.append("High-Priority: User is on known fraud list.")
    
    log_message("HistoryAgent", f"History review complete. {len(flags)} flags found.")
    return {"history_flags": flags}

# --- Parallel Agent 2: The Billing Agent ---
def billing_agent(order_data: dict) -> dict:
    """PARALLEL AGENT 2: Checks for billing/shipping anomalies."""
    log_message("BillingAgent", f"Checking billing for order {order_data['order_id']}...")
    flags = []
    
    # Use the (mocked) IP tool
    ip_info = json.loads(tool_check_ip_location(order_data['ip_address']))
    
    if ip_info["country"] != order_data["billing_country"]:
        flags.append(f"IP/Billing mismatch (IP: {ip_info['country']}, Bill: {order_data['billing_country']})")
        
    if order_data["billing_country"] != order_data["shipping_country"]:
        flags.append("Billing/Shipping country mismatch.")
        
    log_message("BillingAgent", f"Billing review complete. {len(flags)} flags found.")
    return {"billing_flags": flags}

# --- Parallel Agent 3: The Velocity Agent ---
def velocity_agent(order_data: dict) -> dict:
    """PARALLEL AGENT 3: Checks for high-velocity fraud (stateful memory)."""
    ip = order_data['ip_address']
    log_message("VelocityAgent", f"Checking order velocity for IP: {ip}...")
    flags = []
    
    # --- This is the key stateful memory logic ---
    now = datetime.datetime.now()
    
    # READ from Long-Term Memory
    relevant_timestamps = [t for t in MEMORY_BANK["order_timestamps_by_ip"][ip] 
                           if (now - t).total_seconds() < 60]
    
    if len(relevant_timestamps) >= 3:
        flags.append(f"High-Velocity: {len(relevant_timestamps) + 1} orders from this IP in 60s.")
        
    # WRITE to Long-Term Memory
    MEMORY_BANK["order_timestamps_by_ip"][ip].append(now)
    # --- End of stateful logic ---
    
    log_message("VelocityAgent", f"Velocity review complete. {len(flags)} flags found.")
    return {"velocity_flags": flags}

print("Cell 5: Monitor and Parallel Specialist Agents defined.")


Cell 5: Monitor and Parallel Specialist Agents defined.


# 🧠 The Agent Team: Part 2 (The "Brain")
This is the Evaluation Agent that synthesizes the reports and the Publisher Agent that saves the file

In [6]:
# --- Evaluation Agent: The "Brain" ---
def risk_manager_agent(order_data: dict, history_report: dict, billing_report: dict, velocity_report: dict) -> dict:
    """
    EVALUATION AGENT: The "brain."
    Synthesizes reports from the parallel team into one decision.
    Demonstrates: Agent Evaluation, Generative AI (Mocked)
    """
    order_id = order_data['order_id']
    log_message("RiskManagerAgent", f"Synthesizing all reports for order {order_id}...")
    
    all_flags = (history_report["history_flags"] + 
                 billing_report["billing_flags"] + 
                 velocity_report["velocity_flags"])
    
    # --- MOCKED GENERATIVE AI (Evaluation Logic) ---
    risk_score = 0
    justification = ""
    
    if not all_flags:
        risk_score = 0
        justification = "All checks passed. Customer has clean history."
        final_decision = "APPROVE"
    else:
        # Calculate risk score
        if any("High-Priority" in flag for flag in all_flags):
            risk_score = 100
        elif any("High-Velocity" in flag for flag in all_flags):
            risk_score += 50
        
        if any("mismatch" in flag for flag in all_flags):
            risk_score += 30
            
        if any("New customer" in flag for flag in all_flags):
            risk_score += 10
        
        # Generate justification
        justification = f"Risk Score: {risk_score}. Flags: [{'; '.join(all_flags)}]"
        
        if risk_score >= 90:
            final_decision = "REJECT"
        elif risk_score >= 50:
            final_decision = "FLAG_FOR_REVIEW"
        else:
            final_decision = "APPROVE"
            
    log_message("RiskManagerAgent", f"Synthesis complete for {order_id}. Decision: {final_decision}")
    
    # WRITE the final decision to the audit log (Long-Term Memory)
    MEMORY_BANK["decision_log"].append({
        "order_id": order_id,
        "decision": final_decision,
        "score": risk_score,
        "justification": justification,
        "timestamp": datetime.datetime.now().isoformat()
    })
    
    return {"final_decision": final_decision, "justification": justification}


# --- Sequential Agent 3: The Publisher ---
def publisher_agent(order_data: dict, brain_decision: dict):
    """SEQUENTIAL AGENT 3: Uses the Custom Tool to save the final report if flagged."""
    
    if brain_decision["final_decision"] in ["FLAG_FOR_REVIEW", "REJECT"]:
        log_message("PublisherAgent", f"Decision is {brain_decision['final_decision']}. Calling Custom Tool...")
        
        # Use the REAL Custom Tool
        tool_flag_for_manual_review(
            order_id=order_data['order_id'],
            justification=brain_decision["justification"],
            filename="MANUAL_REVIEW_QUEUE.md"
        )
    else:
        log_message("PublisherAgent", "Decision is APPROVE. No file needed.")

print("Cell 6: Evaluation ('Brain') and Publisher Agents defined.")


Cell 6: Evaluation ('Brain') and Publisher Agents defined.


# 🔄 The "Daemon" Loop (Main Function)
This is the main "Loop Agent" that simulates a server, processing a queue of new orders

In [7]:
def run_transaction_shield_swarm(loop_delay_seconds: int):
    """
    Main function to run the agent swarm (Loop Agent).
    Simulates a "daemon" processing a queue of transactions.
    Demonstrates: Loop, Long-Running Ops (Pause/Resume), Sequential, Parallel
    """
    
    log_message("TransactionShield", "Swarm is active. Monitoring order queue...")
    
    # --- MOCKED ORDER QUEUE ---
    # This simulates a live stream of new orders.
    # Note how they are designed to test different fraud rules.
    new_order_queue = [
        # 1. A good, clean order from a known customer
        {"order_id": 1001, "customer_id": "user_77788", "amount": 50.00, 
         "ip_address": "1.2.3.4", "billing_country": "USA", "shipping_country": "USA"},
        
        # 2. A high-risk order (mismatch + new user)
        {"order_id": 1002, "customer_id": "user_00001", "amount": 800.00, 
         "ip_address": "9.8.7.6", "billing_country": "NIGERIA", "shipping_country": "USA"},
        
        # 3. A velocity-fraud order (same IP as #1002, 1s later)
        {"order_id": 1003, "customer_id": "user_00002", "amount": 1200.00, 
         "ip_address": "9.8.7.6", "billing_country": "NIGERIA", "shipping_country": "USA"},
        
        # 4. Another velocity-fraud order (same IP, 1s later)
        {"order_id": 1004, "customer_id": "user_00003", "amount": 950.00, 
         "ip_address": "9.8.7.6", "billing_country": "NIGERIA", "shipping_country": "USA"},
        
        # 5. An order from a known fraudster
        {"order_id": 1005, "customer_id": "user_12345", "amount": 10.00, 
         "ip_address": "5.6.7.8", "billing_country": "USA", "shipping_country": "USA"},
    ]
    
    # --- Clear alert file at the start ---
    alert_filename = "MANUAL_REVIEW_QUEUE.md"
    if os.path.exists(alert_filename):
        os.remove(alert_filename)
        log_message("TransactionShield", "Cleared old review file.")

    # --- Phase 1: The "Daemon" (Loop) ---
    loop_count = 0
    while new_order_queue:
        loop_count += 1
        log_message("TransactionShield", f"--- Starting Loop {loop_count} ---")
        
        # Get the next item from the queue
        order = new_order_queue.pop(0)
        
        # --- Phase 2: Planning (Sequential) ---
        planned_task = monitor_agent(order)
        
        # --- Phase 3: Parallel Execution ---
        log_message("MonitorAgent", f"Spawning parallel agents for order {order['order_id']}...")
        
        report_part_1 = history_agent(planned_task["order_data"])
        report_part_2 = billing_agent(planned_task["order_data"])
        report_part_3 = velocity_agent(planned_task["order_data"])
        
        # Combine parallel results
        combined_reports = {**report_part_1, **report_part_2, **report_part_3}
        
        # --- Phase 4: Evaluation (Sequential "Brain") ---
        brain_decision = risk_manager_agent(
            order_data=planned_task["order_data"],
            history_report=report_part_1,
            billing_report=report_part_2,
            velocity_report=report_part_3
        )
        
        # --- Phase 5: Publishing (Custom Tool) ---
        publisher_agent(planned_task["order_data"], brain_decision)
        
        # --- Phase 6: LONG-RUNNING OPERATION (PAUSE/RESUME) ---
        if new_order_queue:
            log_message("TransactionShield", f"Operation complete. Pausing for {loop_delay_seconds}s...")
            time.sleep(loop_delay_seconds)
            log_message("TransactionShield", "Resuming operation. Checking for new orders...")
    
    log_message("TransactionShield", "All orders processed. Swarm is shutting down.")

print("Cell 7: Main 'Daemon' loop function defined.")


Cell 7: Main 'Daemon' loop function defined.


# 🚀 Run the Swarm!
This cell will execute the entire swarm. You will see the logs print out in real-time as it processes all five (mocked) orders.

In [8]:
# --- Configuration ---\
LOOP_DELAY_SECONDS = 1 # How long to "pause" (1s for demo)

# --- Clear the memory from previous runs (but keep the known fraud users!) ---
MEMORY_BANK["order_timestamps_by_ip"] = defaultdict(list)
MEMORY_BANK["decision_log"] = []
MEMORY_BANK["alerts_fired"] = 0
print("--- Cleared Memory Bank history for new run ---")

# --- Run the entire agent swarm ---\
run_transaction_shield_swarm(
    loop_delay_seconds=LOOP_DELAY_SECONDS
)

print("\n\n" + "="*40)
print("✅ SWARM RUN COMPLETE ✅")
print(f"Total Alerts Generated: {MEMORY_BANK['alerts_fired']}")
print("="*40)


--- Cleared Memory Bank history for new run ---
[2025-11-25 19:26:08] [TransactionShield] [INFO]: Swarm is active. Monitoring order queue...
[2025-11-25 19:26:08] [TransactionShield] [INFO]: --- Starting Loop 1 ---
[2025-11-25 19:26:08] [MonitorAgent] [INFO]: New order detected: 1001. Passing to specialists.
[2025-11-25 19:26:08] [MonitorAgent] [INFO]: Spawning parallel agents for order 1001...
[2025-11-25 19:26:08] [HistoryAgent] [INFO]: Checking history for user user_77788...
[2025-11-25 19:26:08] [CustomerDBTool] [INFO]: Looking up history for: user_77788
[2025-11-25 19:26:08] [HistoryAgent] [INFO]: History review complete. 0 flags found.
[2025-11-25 19:26:08] [BillingAgent] [INFO]: Checking billing for order 1001...
[2025-11-25 19:26:08] [GeoIPTool] [INFO]: Checking location for IP: 1.2.3.4
[2025-11-25 19:26:08] [BillingAgent] [INFO]: Billing review complete. 0 flags found.
[2025-11-25 19:26:08] [VelocityAgent] [INFO]: Checking order velocity for IP: 1.2.3.4...
[2025-11-25 19:26:08

# 📊 Review: Long-Term Memory
Let's inspect the MEMORY_BANK. This proves our agents were reading and writing to memory.

In [9]:
print("Inspecting Long-Term Memory...")
print(f"\n--- Decision Log (Written by 'Brain' Agent) ---")
print(f"Total decisions logged: {len(MEMORY_BANK['decision_log'])}")
print(json.dumps(MEMORY_BANK['decision_log'], indent=2))

print(f"\n--- IP Velocity Log (Written by 'Velocity' Agent) ---")
# We just print the IP that had multiple orders
print(f"Orders logged for IP '9.8.7.6': {len(MEMORY_BANK['order_timestamps_by_ip']['9.8.7.6'])}")


Inspecting Long-Term Memory...

--- Decision Log (Written by 'Brain' Agent) ---
Total decisions logged: 5
[
  {
    "order_id": 1001,
    "decision": "APPROVE",
    "score": 0,
    "justification": "All checks passed. Customer has clean history.",
    "timestamp": "2025-11-25T19:26:08.730689"
  },
  {
    "order_id": 1002,
    "decision": "APPROVE",
    "score": 30,
    "justification": "Risk Score: 30. Flags: [Billing/Shipping country mismatch.]",
    "timestamp": "2025-11-25T19:26:09.731214"
  },
  {
    "order_id": 1003,
    "decision": "APPROVE",
    "score": 40,
    "justification": "Risk Score: 40. Flags: [New customer (higher risk); Billing/Shipping country mismatch.]",
    "timestamp": "2025-11-25T19:26:10.731701"
  },
  {
    "order_id": 1004,
    "decision": "APPROVE",
    "score": 40,
    "justification": "Risk Score: 40. Flags: [New customer (higher risk); Billing/Shipping country mismatch.]",
    "timestamp": "2025-11-25T19:26:11.732189"
  },
  {
    "order_id": 1005,
    

# 🔔 Review: The Final "Flagged" File
This is the final, prize-winning output. We will print the contents of the MANUAL_REVIEW_QUEUE.md file that was created by our Custom Tool. This is the file the human team would use.

In [10]:
print("Inspecting 'MANUAL_REVIEW_QUEUE.md' file (created by Custom Tool)...")
print("="*80 + "\n")

alert_filename = "MANUAL_REVIEW_QUEUE.md"
try:
    with open(alert_filename, 'r') as f:
        print(f.read())
except FileNotFoundError:
    print(f"--- No file found. (This is OK if no orders were flagged) ---")



Inspecting 'MANUAL_REVIEW_QUEUE.md' file (created by Custom Tool)...


-------------------------------------------
###  flagged FOR MANUAL REVIEW
**Order ID:** 1005
**Justification:** Risk Score: 110. Flags: [New customer (higher risk); High-Priority: User is on known fraud list.]
-------------------------------------------


